#  Дрыц-тыц холодильник и шкафы

## Описание


### Как всё будет работать

Заходим в бот телеграмм
Отправляем голосове сообщение, в коором перечисляем, что есть в холодильнике и на кухне в шкафах
Сообщение распознаётся и выдаётся текст (и запрос на подтверждение правильности распознавания)
ДАлее выводится список подходящих рецептов с гиперссылками или же сообщение о том, что ничего не нашлось

### Технические моменты

**Сервер**
* Сначала попробовать работу MVP на Google Colab, потом на Raspberry Pi
* Gjnjv переложить, например, в Yandex Cloud, reg.ru облако и др.
* Как выкладывать на облако - см. курс на Stepik про телеграмм-бот для создания канала новостей

**Распознавание речи**
* отправлять голосове сообщение удобнее, чем стоаять и печатать
* см. проект `telegram_bot_voice_to_text`

**Взаимодействие с пользователем**
* Спросить правльно ли распознался список продуктов 
    - Вывести нумерованный список.
    - Если неправильно, то дать возможность пользователю откорректировать ингридиент, набрав номер в списке и введя новое название.
* Спросить про варианты подбора рецепта: 
    - полное совпадение;
    - недостаток 1 иингридиента;
    - недостаток 2 ингридиентов.

**Логотип**
* Логотип бота сгенерировать в MidJourney / Kandinsy / Шедеврум 

**Данные**
* набрать базу из say7 и т.п. - потом на них законно ссылаться без нарушеняи авторских прав
* можно даже ей предложить сотрудничество - она рекламирует бот, а внём только её рецепты

**Тесты**
* писать заранее - так приучать себя к test driven development (на этапе MVP в Google Colab - это будут простые assert statements или вообще простая проверка на равенство в ячейках)

**Потенциальные проблемы**
* подход с созданием `set` из слов - как быть с названиями ингридиентов из 2 и более слов типа `маринованные огурцы`, `лук репчатый`? Приводить всё к именительному падежу с помощью какой-то библиотеки? 
* Как искать искать игридиены-заменители типа `лук`, `лук репчатый`? Как не спутать `лук` с `перья лука`?

## Библиотеки

In [198]:
import requests
import time
import random
from bs4 import BeautifulSoup
import pymorphy2
import pandas as pd
from sqlalchemy import create_engine, text

morph = pymorphy2.MorphAnalyzer()

## Данные

In [109]:
# !pip install pymorphy2

### План
1. Составляем список ссылок на все рецепты с сайта
2. Проходимся по каждой ссылке и добываем оттуда ингридиенты и их количество.
3. Склываем в таблицу (mySQL) название id-блюда-ссылка-массив с ингридиентами 

### Составляем список ссылок на все рецепты с сайта

In [92]:
import requests
from bs4 import BeautifulSoup

def get_recipe_links(url):
    """
    Extracts recipe links from a given webpage.

    Args:
    - url (str): The URL of the webpage.

    Returns:
    - list: A list of recipe links found on the webpage.
    """
    # Ensure that 'url' is used instead of an undeclared variable 'page'
    response = requests.get(url)

    # Use try-except block to handle potential exceptions during parsing
    try:
        soup = BeautifulSoup(response.text, 'html.parser')
    except Exception as e:
        print(f"Error parsing HTML: {e}")
        return []

    links = []
    for link in soup.find_all('a'):
        recipe_link = link.get('href')

        # Ensure that 'https' is properly checked and 'recipe' is in the URL
        if recipe_link and 'https' in recipe_link and 'recipe' in recipe_link:
            links.append(recipe_link)

    return links

In [94]:
%%time

# первая страница с ссылками на рецепты отличается от последующих
url = 'https://www.say7.info/cook/'
recipe_links = get_recipe_links(url)

# далее принцип формирования ссылок понятен: по 20 на каждой странице
for n in range(20, 1400, 20):
    url = f'https://www.say7.info/cook/linkz_start-{n}.html'
    recipe_links += get_recipe_links(url)

# записываем полученный список ссылок на рецепты в файл
with open('../data/recipes.txt', 'w') as f:
    for url in recipe_links:
        f.write(url+'\n')

# В первый раз всё прошло за 36 c

Wall time: 1min 9s


In [152]:
# читаем файл со списком ссылок
with open('../data/recipes.txt') as f:
    recipe_urls = f.readlines()

# убираем символы перевода каретки из ссылок
recipe_urls = list(map(lambda x: x.strip(), recipe_links))

### Список ингридиентов для каждого рецепта

In [162]:
def get_recipe_name_n_ingredients(recipe_url, headers=None):
    ingredients = []

    # Set headers to mimic a browser
    if headers is None:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    try:
        response = requests.get(recipe_url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {recipe_url}: {e}")
        return {'recipe_name': None, 'ingredients': []}

    soup = BeautifulSoup(response.text, 'html.parser')
    recipe_name = soup.title.text  # название рецепта

    # Extract ingredients logic here
    for item in soup.find_all('li', class_='p-ingredient'):
        ingredients.append(item.text)

    return {'recipe_url': recipe_url, 'recipe_name': recipe_name, 'ingredients': ingredients}


In [170]:
%%time
i = 0
recipe_names = []
ingredients = []

for recipe_url in recipe_urls:
    recipe_attrs = get_recipe_name_n_ingredients(recipe_url)
    recipe_names.append(recipe_attrs['recipe_name'])
    ingredients.append(recipe_attrs['ingredients'])
    print(i, recipe_url, recipe_attrs['recipe_name'])
    
    # делаем паузу, чтоб не сильно атаковать сервер
    random_pause = random.uniform(0.5, 10.0)
    time.sleep(random_pause)
    i += 1

0 https://www.say7.info/cook/recipe/1590-Tort-Tri.html Торт «Три шоколада»
1 https://www.say7.info/cook/recipe/56-Salat-Mimoza.html Салат «Мимоза»
2 https://www.say7.info/cook/recipe/935-Pelmeni.html Пельмени
3 https://www.say7.info/cook/recipe/60-YAblochnyiy-pirog.html Яблочный пирог
4 https://www.say7.info/cook/recipe/1521-Brusketta-tuncom.html Брускетта с тунцом и помидорами
5 https://www.say7.info/cook/recipe/1470-Lazanya.html Лазанья с кабачками и фаршем
6 https://www.say7.info/cook/recipe/1405-Pletenka.html Плетенка с шоколадной пастой
7 https://www.say7.info/cook/recipe/1008-Salat-Cezar.html Салат «Цезарь»
8 https://www.say7.info/cook/recipe/1599-Glintveyn.html Глинтвейн с брусникой
9 https://www.say7.info/cook/recipe/541-Blinyi-drozhzhevyie.html Блины дрожжевые
10 https://www.say7.info/cook/recipe/245-Ryiba-zharennaya.html Рыба, жаренная в кляре
11 https://www.say7.info/cook/recipe/1363-Pechene.html Печенье «Бананчики»
12 https://www.say7.info/cook/recipe/246-Syirnyiy-sup.html 

96 https://www.say7.info/cook/recipe/1067-Krem-karamel.html Крем-карамель
97 https://www.say7.info/cook/recipe/1429-Kurinaya-pechen.html Куриная печень с грибами в сметанно-горчичном соусе
98 https://www.say7.info/cook/recipe/285-Kurica-gril.html Курица-гриль
99 https://www.say7.info/cook/recipe/1184-Malosolnaya-semga.html Малосольная семга
100 https://www.say7.info/cook/recipe/913-Pasta-Karbonara.html Паста «Карбонара»
101 https://www.say7.info/cook/recipe/1308-Pirog-yablokami.html Пирог с яблоками (а-ля чизкейк)
102 https://www.say7.info/cook/recipe/1293-Pirozhnoe-Ptiche.html Пирожное «Птичье молоко»
103 https://www.say7.info/cook/recipe/1374-Savaren.html Саварен
104 https://www.say7.info/cook/recipe/1269-Sehndvich-bekonom.html Сэндвич с беконом и яичницей
105 https://www.say7.info/cook/recipe/1438-Tort-Krasnyiy.html Торт «Красный бархат»
106 https://www.say7.info/cook/recipe/889-KHolodec.html Холодец
107 https://www.say7.info/cook/recipe/1380-Fugas.html Фугас
108 https://www.say7.in

Error fetching https://www.say7.info/cook/recipe/694-YAblochno-morkovnyie.html: ('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))
184 https://www.say7.info/cook/recipe/694-YAblochno-morkovnyie.html None
Error fetching https://www.say7.info/cook/recipe/1403-Panna-kotta.html: ('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))
185 https://www.say7.info/cook/recipe/1403-Panna-kotta.html None
186 https://www.say7.info/cook/recipe/1214-Ovoschi-yaycom.html Овощи с яйцом, запеченные в булочке
187 https://www.say7.info/cook/recipe/683-Pirog-kapustoy.html Пирог с капустой
188 https://www.say7.info/cook/recipe/1357-Pirog-chernoy.html Пирог с черной смородиной
189 https://www.say7.info/cook/recipe/1320-Picca.html Пицца с морепродуктами
190 https://www.say7.info/cook/recipe/650-Moreproduktyi.html Морепродукты с рисом
191 https

264 https://www.say7.info/cook/recipe/1326-Imbirnoe-pechene.html Имбирное печенье
265 https://www.say7.info/cook/recipe/74-Kurica.html Курица с картофелем, запеченные под майонезом
266 https://www.say7.info/cook/recipe/121-Kurica-ovoschami.html Курица с овощами, тушенная в фольге
267 https://www.say7.info/cook/recipe/1385-Makovyie.html Маковые треугольники
268 https://www.say7.info/cook/recipe/1295-Napoleon-lenivyiy.html Наполеон ленивый
269 https://www.say7.info/cook/recipe/1495-Okun-zapechennyiy.html Окунь, запеченный под сметанно-сырным соусом
Error fetching https://www.say7.info/cook/recipe/1307-Morkovno-apelsino.html: ('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))
270 https://www.say7.info/cook/recipe/1307-Morkovno-apelsino.html None
271 https://www.say7.info/cook/recipe/1370-Morkovnyiy-pirog.html Морковный пирог с абрикосами и грецкими орехами
272 https://www.say7.info/cook/recipe/1261-Moro

351 https://www.say7.info/cook/recipe/1354-Gorbusha.html Горбуша, запеченная с картофелем, грибами и сыром
352 https://www.say7.info/cook/recipe/666-Kryilyishki-sousom.html Крылышки с соусом из сыра с благородной плесенью
353 https://www.say7.info/cook/recipe/704-Krekeryi-avokado.html Крекеры с авокадо и помидорами черри
354 https://www.say7.info/cook/recipe/604-Kotletyi.html Котлеты из индейки, фаршированные грибами
355 https://www.say7.info/cook/recipe/495-Kurinyie-bedryishki.html Куриные бедрышки, запеченные с сыром
356 https://www.say7.info/cook/recipe/404-Kurica.html Курица, фаршированная грибами
357 https://www.say7.info/cook/recipe/1290-Pirog-testa.html Пирог из теста фило с вишней
358 https://www.say7.info/cook/recipe/1397-Pirog-vyalenyimi.html Пирог с вялеными томатами и куриным филе
359 https://www.say7.info/cook/recipe/1334-Pirog.html Пирог с мандаринами
360 https://www.say7.info/cook/recipe/1014-Picca-Kalcone.html Пицца «Кальцоне»
361 https://www.say7.info/cook/recipe/1091-

...

1022 https://www.say7.info/cook/recipe/783-Pshennaya-kasha.html Пшенная каша с тыквой в горшочке
1023 https://www.say7.info/cook/recipe/289-Salat-kapustoy.html Салат с капустой, яблоком и изюмом
1024 https://www.say7.info/cook/recipe/520-Sakharnaya-glazur.html Сахарная глазурь
1025 https://www.say7.info/cook/recipe/440-Sloenyie-pirozhki.html Слоеные пирожки с мясом и помидорами
1026 https://www.say7.info/cook/recipe/30-Tvorozhnyie-palochki.html Творожные палочки
1027 https://www.say7.info/cook/recipe/755-Fetuchini.html Фетучини с телятиной и грибами под сливочным соусом
1028 https://www.say7.info/cook/recipe/700-Cukkini.html Цуккини, фаршированные фетой
1029 https://www.say7.info/cook/recipe/507-YAichnica-vetchinoy.html Яичница с ветчиной и помидорами
1030 https://www.say7.info/cook/recipe/493-Baklazhanyi.html Баклажаны с грецкими орехами
1031 https://www.say7.info/cook/recipe/73-Baklazhanyi.html Баклажаны, запеченные с помидорами и сыром
1032 https://www.say7.info/cook/recipe/1348-Gor

Error fetching https://www.say7.info/cook/recipe/238-Sup-KHarcho.html: ('Connection aborted.', OSError(0, 'Error'))
1085 https://www.say7.info/cook/recipe/238-Sup-KHarcho.html None
1086 https://www.say7.info/cook/recipe/845-Syirnyiy-sup.html Сырный суп с белыми грибами
Error fetching https://www.say7.info/cook/recipe/746-Sdobnyie-bulochki.html: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
1087 https://www.say7.info/cook/recipe/746-Sdobnyie-bulochki.html None
Error fetching https://www.say7.info/cook/recipe/997-Farshirovannyiy.html: ('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))
1088 https://www.say7.info/cook/recipe/997-Farshirovannyiy.html None
Error fetching https://www.say7.info/cook/recipe/447-Tvorozhnyiy-desert.html: ('Connection aborted.', OSError(0, 'Error'))
1089 https://www.say7.info/cook/recipe/447-Tvorozhnyiy-desert.html None
1090 https:/

1164 https://www.say7.info/cook/recipe/985-Drozhzhevyie-pirozhki.html Дрожжевые пирожки на кефире (тесто для дрожжевых пирожков)
1165 https://www.say7.info/cook/recipe/1141-SHokoladnoe-pechene.html Шоколадное печенье с курагой
1166 https://www.say7.info/cook/recipe/1176-Bulgur-tomatami.html Булгур с томатами и луком (гарнир)
1167 https://www.say7.info/cook/recipe/225-Ryiba-tushennaya.html Рыба, тушенная с овощами в фольге
1168 https://www.say7.info/cook/recipe/496-Pryaniki.html Пряники кофейно-шоколадные
1169 https://www.say7.info/cook/recipe/867-Okroshka.html Окрошка
1170 https://www.say7.info/cook/recipe/445-Omlet-vetchinoy.html Омлет с ветчиной и помидорами
1171 https://www.say7.info/cook/recipe/579-Sup-krapivoy.html Суп с крапивой
1172 https://www.say7.info/cook/recipe/996-Sup-nutom.html Суп с нутом и бараниной
1173 https://www.say7.info/cook/recipe/433-Sup-chechevicey.html Суп с чечевицей, беконом и сухариками
1174 https://www.say7.info/cook/recipe/1093-Syirnoe-pechene.html Сырное

1250 https://www.say7.info/cook/recipe/931-Kartofelno-gribnyi.html Картофельно-грибные котлеты со сметанным соусом
1251 https://www.say7.info/cook/recipe/1322-Kokosovyie-keksyi.html Кокосовые кексы с кокосовым кремом
1252 https://www.say7.info/cook/recipe/629-Maffinyi-yagodoy.html Маффины с ягодой
1253 https://www.say7.info/cook/recipe/1063-Pirog-sloenogo.html Пирог из слоеного теста с фаршем и овощами
1254 https://www.say7.info/cook/recipe/1138-Sup-gribami.html Суп с грибами, куриным филе и сливками
1255 https://www.say7.info/cook/recipe/1088-Tartyi-tyikvoy.html Тарты с тыквой и яблоками
1256 https://www.say7.info/cook/recipe/21-SHtrudeli.html Штрудели
1257 https://www.say7.info/cook/recipe/886-Buterbrodyi.html Бутерброды «Змейка»
1258 https://www.say7.info/cook/recipe/1190-Kabachki.html Кабачки с помидорами под сметанным соусом и сыром
1259 https://www.say7.info/cook/recipe/177-Kartofel.html Картофель, обжаренный с луком и морковью
1260 https://www.say7.info/cook/recipe/164-Kambala.h

1338 https://www.say7.info/cook/recipe/465-Vafelnyie.html Вафельные корзиночки с белковым кремом
1339 https://www.say7.info/cook/recipe/581-Zapekanka-risom.html Запеканка с рисом, яблоками и изюмом
1340 https://www.say7.info/cook/recipe/1198-Karamelnyiy-sous.html Карамельный соус
1341 https://www.say7.info/cook/recipe/1153-Maffinyi.html Маффины с клубникой и базиликом на клубничном йогурте
1342 https://www.say7.info/cook/recipe/981-Salat.html Салат с баклажанами, фунчозой и говядиной
1343 https://www.say7.info/cook/recipe/1143-Salat.html Салат с охотничьими колбасками и сухариками
1344 https://www.say7.info/cook/recipe/307-Sloenyie-rogaliki.html Слоеные рогалики с орехами
1345 https://www.say7.info/cook/recipe/949-Tefteli.html Тефтели с моцареллой в томатном соусе
1346 https://www.say7.info/cook/recipe/897-Citrusovyie.html Цитрусовые пирожные «Валентинки» с фруктами
1347 https://www.say7.info/cook/recipe/208-Gus-farshirovannyiy.html Гусь фаршированный
1348 https://www.say7.info/cook/re

```
ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))
```

Here is what I think happens: I'm trying to scrap data. The webserver does not like that and stops talking to my PC. The people owning the webserver have put measures into place that prevent random people from scraping all their stuff.

In [171]:
df = pd.DataFrame(
    {
        'url': recipe_urls,
        'recipe_name': recipe_names,
        'ingredients': ingredients
    }
)

In [174]:
df.to_excel('../data/base.xlsx')

In [175]:
# df.to_csv('../data/base.csv')

In [183]:
# собираем то, что не получилось в первый раз
recipe_urls = []
recipe_names = []
ingredients = []
i = 0

for recipe_url in df[df['recipe_name'].isna()]['url']:
    recipe_attrs = get_recipe_name_n_ingredients(recipe_url)
    recipe_urls.append(recipe_attrs['recipe_url'])
    recipe_names.append(recipe_attrs['recipe_name'])
    ingredients.append(recipe_attrs['ingredients'])
    print(i, recipe_url, recipe_attrs['recipe_name'])
    
    # делаем паузу
    random_pause = random.uniform(0.5, 5)
    time.sleep(random_pause)
    i += 1

1400 https://www.say7.info/cook/recipe/207-Salat-Granatovyiy.html Салат «Гранатовый браслет»
1401 https://www.say7.info/cook/recipe/744-Zalivnaya-ryiba.html Заливная рыба
1402 https://www.say7.info/cook/recipe/1193-Tertyiy-yablochnyiy.html Тертый яблочный пирог
1403 https://www.say7.info/cook/recipe/694-YAblochno-morkovnyie.html Яблочно-морковные оладьи
1404 https://www.say7.info/cook/recipe/1403-Panna-kotta.html Панна котта с малиновым желе
1405 https://www.say7.info/cook/recipe/1485-Morozhenoe.html Мороженое клубничное «Тирамису»
1406 https://www.say7.info/cook/recipe/1121-Sochni.html Сочни
1407 https://www.say7.info/cook/recipe/1328-EHkleryi.html Эклеры с апельсиновым кремом
1408 https://www.say7.info/cook/recipe/1307-Morkovno-apelsino.html Морковно-апельсиновый пирог с сиропом
1409 https://www.say7.info/cook/recipe/437-Tort-Morkovnyiy.html Торт «Морковный»
1410 https://www.say7.info/cook/recipe/1041-Kotletyi.html Котлеты, запеченные в духовке под грибным соусом
1411 https://www.say

In [184]:
df_to_add = pd.DataFrame(
    {
        'url': recipe_urls,
        'recipe_name': recipe_names,
        'ingredients': ingredients
    }
)

In [190]:
df.dropna(subset=['recipe_name'], inplace=True)
df = df.append(df_to_add)

In [191]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1394 entries, 0 to 52
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   url          1394 non-null   object
 1   recipe_name  1394 non-null   object
 2   ingredients  1394 non-null   object
dtypes: object(3)
memory usage: 43.6+ KB


In [194]:
df.to_csv('../data/base.csv', index=None)
df.to_excel('../data/base.xlsx', index=None)

### Преобазование списка ингридиентов (начальная форма) и создание правил для поиска и замены

In [196]:
df = pd.read_excel('../data/database.xlsx')

In [197]:
df.head()

,url,recipe_name,ingredients
0,https://www.say7.info/cook/recipe/1590-Tort-Tr...,Торт «Три шоколада»,"['3 яйца (небольших)', '100\xa0г сахара', '70\..."
1,https://www.say7.info/cook/recipe/56-Salat-Mim...,Салат «Мимоза»,['200\xa0г консервированной в\xa0масле рыбы (л...
2,https://www.say7.info/cook/recipe/935-Pelmeni....,Пельмени,"['250\xa0мл молока', '1 яйцо', '1\xa0ст.л. рас..."
3,https://www.say7.info/cook/recipe/60-YAblochny...,Яблочный пирог,"['1\xa0кг яблок (желательно, кислых или\xa0кис..."
4,https://www.say7.info/cook/recipe/1521-Brusket...,Брускетта с тунцом и помидорами,"['150\xa0г багета', '150\xa0г тунца (я\xa0испо..."


In [ ]:
def make_initial_form(ingredient):
    """
    ingredient (str): name of the ingredient
    """
    ingredient_initial = [morph.parse(word)[0].normal_form for word in ingredient.split()]
    return ' '.join(ingredient_initial)



In [146]:
# Example ingredients
ingredients = [
    "лука репчатого", "куриных бедрышек", "помидоров", "зелени по вкусу",
    "очищенных кедровых орехов", "зубчика чеснока", "растительного масла",
    "куриного филе", "лимонного сока", "миндаля", "сахарной пудры (или сахара)"        
]

ingredients_initial = [make_initial_form(ingredient) for ingredient in ingredients]
ingredients_initial

['лука репчатый',
 'куриный бедрышка',
 'помидор',
 'зелень по вкус',
 'очистить кедровый орех',
 'зубчик чеснок',
 'растительный масло',
 'куриный филе',
 'лимонный сок',
 'миндаль',
 'сахарный пудра (ить сахара)']

### Создание базы данных SQLite

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html

In [207]:
engine = create_engine('sqlite://', echo=False)
df.to_sql(name='recipes', con=engine)

TypeError: __init__() got multiple values for argument 'schema'

In [ ]:
# TypeError: __init__() got multiple values for argument 'schema'
#  this is an indication of a mismatch between pandas and SQLAlchemy version.

In [208]:
!pip install --upgrade pandas

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Отказано в доступе: 'C:\\Anaconda3\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2


In [ ]:
pip3 install --upgrade sqlalchemy

In [ ]:
# with engine.connect() as conn:
#     conn.execute(text("SELECT * FROM users")).fetchall()